In [1]:
import os
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input # type: ignore
from tensorflow.keras.preprocessing.image import load_img, img_to_array # type: ignore
from tensorflow.keras.models import Model # type: ignore
import cv2 as cv
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 
from scipy.io import loadmat
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import median_absolute_error
from sklearn.datasets import fetch_openml
from sklearn.impute import SimpleImputer
import re
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing import image # type: ignore
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


c:\Users\ASUS\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
split_df =  pd.read_csv('split_df.csv',encoding='utf-8-sig')

In [3]:
# load the model
model = VGG16()
# restructure the model
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
# summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [61]:
# for layer in model.layers[:15]: # مشان نلغي اول 15 طبقة
#     layer.trainable = False

In [158]:
# # features = {}
# # directory = 'Data/'
# def extract_orb_features(image_path):
# # for image in os.listdir(directory):
# #     image_path = directory+image
    
#     # load the image
#     img = load_img(image_path, target_size=(224, 224))
#     # convert pixel to numpy array
#     img = img_to_array(img)
#     # reshape the image for the model
#     img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
#     # preprocess the image
#     img = preprocess_input(img)
#     # extract features
#     feature = model.predict(img, verbose=0)
#     # store feature
#     # features[image_path] = feature
#     return feature.flatten()

In [4]:
split_df = split_df.dropna(subset=['Image'])

In [5]:
len(split_df)

86

In [6]:
model = VGG16(weights='imagenet', include_top=False)

def extract_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = model.predict(img_array)
    return features.flatten()

split_df['VGG16_features'] = list(split_df['Image'].apply(extract_features))

1/1 [==============================] - 0s 120ms/step


In [7]:
split_df

,Table 1,Spectacles,Age,Gender,Name,id,Eye,VA,CDR,Diagnosis,IOP,Image,VGG16_features
0,زرق أيمن تم معالجته جراحيا,0.0,64,0,شاهين ميرو,1.0,0,0.6,0.5,0,14.0,Data/SHAHEN_MERO Optic nerve.png_ left.png,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,زرق أيمن تم معالجته جراحيا,0.0,64,0,شاهين ميرو,1.0,1,0.7,0.9,1,23.0,Data/SHAHEN_MERO Optic nerve.png_ left.png,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,زرق متقدم,1.0,62,0,أيمن سمارة,2.0,0,NaN,NaN,1,25.0,Data/SHAHEN_MERO Optic nerve.png_ right.png,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8849216, 0.0,..."
3,زرق متقدم,1.0,62,0,أيمن سمارة,2.0,1,9.0,0.8,1,40.0,Data/SHAHEN_MERO Optic nerve.png_ right.png,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8849216, 0.0,..."
6,زرق مزدوج أشد بالأيمن,1.0,60,0,مصطفى ملا علي,4.0,0,0.2,0.7,1,10.0,Data/AYMAN_SAMARA_20240121_124719_Disc_3D_R_SI...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,زرق أيسر تم معالجته جراحيا,1.0,55,1,زريفة المحمد,43.0,1,NaN,NaN,0,NaN,Data/BASEL_BASH Optic nerve.png_ left.png,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
86,مريض زرقي ثانوي بعد استخراج ساد لا يوجد اصابة ...,0.0,71,0,هيثم قزاز,44.0,0,0.3,0.5,1,15.0,Data/BASEL_BASH Optic nerve.png_ right.png,"[27.140749, 0.0, 0.0, 0.0, 0.0, 0.0, 24.350609..."
87,مريض زرقي ثانوي بعد استخراج ساد لا يوجد اصابة ...,0.0,71,0,هيثم قزاز,44.0,1,0.6,0.7,1,16.0,Data/BASEL_BASH Optic nerve.png_ right.png,"[27.140749, 0.0, 0.0, 0.0, 0.0, 0.0, 24.350609..."
88,لا يوجد زرق,1.0,65,0,حمود غشام,45.0,0,0.5,0.3,0,15.0,Data/NEZAR_HELOU Optic nerve.png_ left.png,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [28]:
len(split_df)

86

In [6]:
len(split_df['VGG16_features'][1])

25088

In [10]:
y_data = split_df['Diagnosis']

In [8]:
features_array = np.array(split_df['VGG16_features'].tolist())

reduced_features_column_names = ['VGG16_feature_' + str(i) for i in range(25088)]
reduced_features_df = pd.DataFrame(features_array, columns=reduced_features_column_names)
# split_df = pd.concat([split_df, reduced_features_df], axis=1)

In [11]:
feature_names = reduced_features_df.columns.tolist()

# Step 3: Train Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(reduced_features_df, y_data)

# Step 4: Determine feature importance
feature_importances = rf.feature_importances_

# Step 5: Create a DataFrame for visualization
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
})

# Step 6: Sort the DataFrame by importance
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# # Step 7: Visualize feature importance
# plt.figure(figsize=(10, 100))
# sns.barplot(x='Importance', y='Feature', data=importance_df)
# plt.title('Feature Importance in Random Forest')
# plt.xlabel('Importance')
# plt.ylabel('Feature')
# plt.show()

In [12]:
new_feauters = importance_df['Feature'][0:50]
reduced_features_df = reduced_features_df[new_feauters]

In [13]:
split_df = pd.concat([split_df, reduced_features_df], axis=1)

In [14]:
split_df

,Table 1,Spectacles,Age,Gender,Name,id,Eye,VA,CDR,Diagnosis,...,VGG16_feature_17507,VGG16_feature_23996,VGG16_feature_21659,VGG16_feature_22676,VGG16_feature_3938,VGG16_feature_8753,VGG16_feature_5705,VGG16_feature_24525,VGG16_feature_12033,VGG16_feature_10455
0,زرق أيمن تم معالجته جراحيا,0.0,64.0,0.0,شاهين ميرو,1.0,0.0,0.6,0.5,0.0,...,21.571554,0.000000,5.290904,15.228388,0.000000,68.878944,0.0,0.00000,7.349987,0.00000
1,زرق أيمن تم معالجته جراحيا,0.0,64.0,0.0,شاهين ميرو,1.0,1.0,0.7,0.9,1.0,...,21.571554,0.000000,5.290904,15.228388,0.000000,68.878944,0.0,0.00000,7.349987,0.00000
2,زرق متقدم,1.0,62.0,0.0,أيمن سمارة,2.0,0.0,NaN,NaN,1.0,...,12.394115,3.053453,6.093272,0.000000,0.000000,44.247169,0.0,0.00000,0.000000,0.00000
3,زرق متقدم,1.0,62.0,0.0,أيمن سمارة,2.0,1.0,9.0,0.8,1.0,...,12.394115,3.053453,6.093272,0.000000,0.000000,44.247169,0.0,0.00000,0.000000,0.00000
6,زرق مزدوج أشد بالأيمن,1.0,60.0,0.0,مصطفى ملا علي,4.0,0.0,0.2,0.7,1.0,...,12.471640,3.030842,4.392025,0.000000,0.000000,34.289696,0.0,0.00000,8.275780,0.34168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,لا يوجد زرق,1.0,65.0,0.0,حمود غشام,45.0,1.0,0.2,0.3,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19.653715,0.000000,0.000000,16.799368,0.000000,57.460518,0.0,0.00000,0.000000,0.00000
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19.653715,0.000000,0.000000,16.799368,0.000000,57.460518,0.0,0.00000,0.000000,0.00000
18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.535872,0.000000,3.759122,0.000000,13.637669,62.785015,0.0,2.15042,0.000000,0.00000


In [7]:
features_array = np.array(split_df['VGG16_features'].tolist())

# تطبيق PCA
num_features = 50  # عدد المكونات الذي تريد تقليلها إلى
pca = PCA(n_components=num_features)
reduced_features = pca.fit_transform(features_array)

# إضافة الخصائص المخفضة إلى DataFrame
reduced_features_column_names = ['PCA_feature_' + str(i) for i in range(num_features)]
reduced_features_df = pd.DataFrame(reduced_features, columns=reduced_features_column_names)
split_df = pd.concat([split_df, reduced_features_df], axis=1)

In [85]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [161]:
split_df['VGG16_features'] = list(split_df['Image'].apply(extract_orb_features))

In [66]:
# feature.tolist()
# features['Data/5725_HASAN_MARAE Optic nerve.png_ left.png']
# store the dictionary using pickle

In [56]:
split_df=split_df.drop(['Table 1','Gender','id','Name','Eye','Age','Spectacles','Image','VGG16_features'],axis=1)
# split_df=split_df.dropna(thresh=len(split_df.columns) - 1)

In [57]:
split_df.dropna(subset=['Diagnosis'], inplace=True)

In [58]:
X_data = split_df.drop(['Diagnosis'],axis=1)
y_data = split_df['Diagnosis']

In [59]:
len(y_data)

86

In [60]:
y_data.isnull().sum()

0

In [61]:
X_data

,VA,CDR,IOP,VGG16_feature_24216,VGG16_feature_21250,VGG16_feature_6119,VGG16_feature_10012,VGG16_feature_9631,VGG16_feature_18998,VGG16_feature_8347,...,VGG16_feature_17507,VGG16_feature_23996,VGG16_feature_21659,VGG16_feature_22676,VGG16_feature_3938,VGG16_feature_8753,VGG16_feature_5705,VGG16_feature_24525,VGG16_feature_12033,VGG16_feature_10455
0,0.6,0.5,14.0,0.130793,8.639184,0.0,3.076178,0.000000,0.00000,26.851395,...,21.571554,0.000000,5.290904,15.228388,0.000000,68.878944,0.0,0.000000,7.349987,0.00000
1,0.7,0.9,23.0,0.130793,8.639184,0.0,3.076178,0.000000,0.00000,26.851395,...,21.571554,0.000000,5.290904,15.228388,0.000000,68.878944,0.0,0.000000,7.349987,0.00000
2,NaN,NaN,25.0,3.275346,8.321835,0.0,0.000000,16.339722,0.00000,12.470031,...,12.394115,3.053453,6.093272,0.000000,0.000000,44.247169,0.0,0.000000,0.000000,0.00000
3,9.0,0.8,40.0,3.275346,8.321835,0.0,0.000000,16.339722,0.00000,12.470031,...,12.394115,3.053453,6.093272,0.000000,0.000000,44.247169,0.0,0.000000,0.000000,0.00000
6,0.2,0.7,10.0,2.004532,9.620230,0.0,41.360752,5.621842,1.55726,12.004295,...,12.471640,3.030842,4.392025,0.000000,0.000000,34.289696,0.0,0.000000,8.275780,0.34168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,NaN,NaN,NaN,4.235251,0.000000,0.0,0.000000,0.000000,0.00000,17.668137,...,5.224127,7.845808,3.499448,18.914507,29.423475,57.597664,0.0,4.246768,14.041794,0.00000
86,0.3,0.5,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,0.6,0.7,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,0.5,0.3,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [267]:
# X_data['VGG16_features'] = X_data['VGG16_features'].apply(lambda x: np.sum(x))

In [268]:
# X_data

,VA,CDR,IOP,VGG16_features
0,0.6,0.5,14.0,2182.982178
1,0.7,0.9,23.0,2182.982178
2,NaN,NaN,25.0,2446.085205
3,9.0,0.8,40.0,2446.085205
6,0.2,0.7,10.0,2685.752197
...,...,...,...,...
85,NaN,NaN,NaN,2640.070801
86,0.3,0.5,15.0,2469.391113
87,0.6,0.7,16.0,2469.391113
88,0.5,0.3,15.0,2839.579834


In [62]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42, shuffle=True)


In [105]:
X_train.dtypes

VA                float64
CDR               float64
IOP               float64
PCA_feature_0     float32
PCA_feature_1     float32
PCA_feature_2     float32
PCA_feature_3     float32
PCA_feature_4     float32
PCA_feature_5     float32
PCA_feature_6     float32
PCA_feature_7     float32
PCA_feature_8     float32
PCA_feature_9     float32
PCA_feature_10    float32
PCA_feature_11    float32
PCA_feature_12    float32
PCA_feature_13    float32
PCA_feature_14    float32
PCA_feature_15    float32
PCA_feature_16    float32
PCA_feature_17    float32
PCA_feature_18    float32
PCA_feature_19    float32
PCA_feature_20    float32
PCA_feature_21    float32
PCA_feature_22    float32
PCA_feature_23    float32
PCA_feature_24    float32
PCA_feature_25    float32
PCA_feature_26    float32
PCA_feature_27    float32
PCA_feature_28    float32
PCA_feature_29    float32
PCA_feature_30    float32
PCA_feature_31    float32
PCA_feature_32    float32
PCA_feature_33    float32
PCA_feature_34    float32
PCA_feature_

In [63]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [64]:
X_train_scaled

array([[-0.39805815,  0.03941426,  0.20873827, ...,  3.21657796,
        -0.50838524,  0.85263969],
       [-0.58110678, -0.35472838, -1.03499393, ..., -0.60324501,
        -0.54865407,  1.07374499],
       [-0.23534825, -0.35472838, -0.24352798, ..., -0.46995998,
        -0.54865407, -0.75849194],
       ...,
       [-0.19467077, -0.11824279, -0.24352798, ..., -0.29822597,
        -0.54865407, -0.75849194],
       [-0.19467077, -0.27589985, -0.58272768, ..., -0.60324501,
         0.38882055, -0.75849194],
       [-0.19467077, -0.35472838,  1.79167017, ...,  0.0054377 ,
         0.10224412,  0.78182695]])

In [65]:
print(np.isnan(X_train_scaled).sum())

220


In [66]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
X_train_scaled = imputer.fit_transform(X_train_scaled)
X_test_scaled = imputer.transform(X_test_scaled)


In [67]:
svm = SVC()

param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
}

svm_grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=10, n_jobs=-1, verbose=2)
svm_grid_search.fit(X_train_scaled, y_train)

print("Best parameters found: ", svm_grid_search.best_params_)
print("Best R2 score: ", svm_grid_search.best_score_)

best_svm = svm_grid_search.best_estimator_
best_svm.fit(X_train_scaled, y_train)

y_train_pred  = best_svm.predict(X_train_scaled)

train_accuracy = accuracy_score(y_train, y_train_pred )
print(f'Accuracy For train: {train_accuracy:.2f}')

y_pred = best_svm.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(classification_report(y_test, y_pred))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))


Fitting 10 folds for each of 64 candidates, totalling 640 fits
Best parameters found:  {'C': 10, 'gamma': 1, 'kernel': 'linear'}
Best R2 score:  0.7476190476190476
Accuracy For train: 0.96
Accuracy: 0.78
Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      0.50      0.67         8
         1.0       0.71      1.00      0.83        10

    accuracy                           0.78        18
   macro avg       0.86      0.75      0.75        18
weighted avg       0.84      0.78      0.76        18

Confusion Matrix:
[[ 4  4]
 [ 0 10]]


In [110]:
clf = svm.SVC(C=0.9,kernel='linear')

clf.fit(X_train_scaled,y_train)

y_pred = clf.predict(X_test_scaled)

# حساب الدقة
accuracy = accuracy_score(y_test, y_pred)

# طباعة الدقة
print(f'accuracy: {accuracy}')

accuracy: 0.6666666666666666


In [111]:
# clf = svm.SVC(C=0.9,kernel='rbf',gamma=2)
clf.set_params(C=1.2,kernel='rbf',gamma=1)## Gamma مدى الليونة

clf.fit(X_train_scaled,y_train)

y_pred = clf.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)

print(f'accuracy: {accuracy}')

accuracy: 0.6666666666666666


In [112]:
# clf = svm.SVC(C=1.0,kernel='poly',degree= 3,gamma=3)
clf.set_params(C=0.9,kernel='poly',degree=1,gamma=40)

clf.fit(X_train_scaled,y_train)

y_pred = clf.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)

print(f'accuracy: {accuracy}')

accuracy: 0.6666666666666666


In [113]:
clf2 = svm.SVC()

clf2.fit(X_train_scaled,y_train)

y_pred = clf2.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)

print(f'accuracy: {accuracy}')

accuracy: 0.6111111111111112
